In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

## imgflip_service.py

In [3]:
import json
import requests
from typing import Optional, Dict, List

class ImgflipService:
    """
    A service class for generating memes using the Imgflip API.
    Documentation: https://api.imgflip.com/
    """
    
    def __init__(self, username: str, password: str):
        """
        Initialize the meme generator service.
        
        Args:
            username: Imgflip username
            password: Imgflip password
        """
        self.base_url = "https://api.imgflip.com"
        self.username = username
        self.password = password

    def get_popular_memes(self, box_count: Optional[int] = None) -> List[Dict]:
        """
        Get a list of popular meme templates from Imgflip.
        
        Args:
            box_count: Optional filter for number of text boxes
            
        Returns:
            List of dictionaries containing meme template information
        """
        try:
            response = requests.get(f"{self.base_url}/get_memes")
            response.raise_for_status()
            
            data = response.json()
            if data["success"]:
                memes = data["data"]["memes"]
                if box_count is not None:
                    memes = [meme for meme in memes if meme["box_count"] == box_count]
                return memes
            else:
                raise Exception(f"Failed to get memes: {data['error_message']}")
                
        except requests.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

    def get_two_box_memes(self) -> List[Dict]:
        """
        Get a list of meme templates that have exactly two text boxes.
        
        Returns:
            List of dictionaries containing meme template information
        """
        return self.get_popular_memes(box_count=2)

    def create_meme(
        self,
        template_id: str,
        top_text: str,
        bottom_text: str,
        max_font_size: Optional[int] = None
    ) -> Dict:
        """
        Generate a meme using the specified template and two text boxes.
        
        Args:
            template_id: The ID of the meme template to use
            top_text: Text for the top box
            bottom_text: Text for the bottom box
            max_font_size: Optional maximum font size for the text
            
        Returns:
            Dictionary containing the URL of the generated meme
        """
        # Prepare the request payload
        payload = {
            "template_id": template_id,
            "username": self.username,
            "password": self.password,
            "boxes[0][text]": top_text,
            "boxes[1][text]": bottom_text
        }
        
        if max_font_size:
            payload["max_font_size"] = max_font_size

        try:
            response = requests.post(f"{self.base_url}/caption_image", data=payload)
            response.raise_for_status()
            
            data = response.json()
            if data["success"]:
                return {
                    "url": data["data"]["url"],
                    "page_url": data["data"]["page_url"]
                }
            else:
                raise Exception(f"Failed to generate meme: {data['error_message']}")
                
        except requests.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

    def find_template_by_name(self, name: str, two_boxes_only: bool = False) -> Optional[Dict]:
        """
        Find a meme template by name (case-insensitive partial match).
        
        Args:
            name: Name of the meme template to find
            two_boxes_only: If True, only return templates with exactly two text boxes
            
        Returns:
            Dictionary containing template information or None if not found
        """
        templates = self.get_two_box_memes() if two_boxes_only else self.get_popular_memes()
        name = name.lower()
        
        for template in templates:
            if name in template["name"].lower():
                return template
                
        return None

    def list_two_box_meme_names(self) -> List[str]:
        """
        Get a list of names of all available two-box meme templates.
        
        Returns:
            List of meme template names
        """
        templates = self.get_two_box_memes()
        return [template["name"] for template in templates]

## meme_creation_processor.py

In [7]:
from groq import Groq

In [8]:
imgflip_service = ImgflipService(os.environ.get('IMGFLIP_USERNAME'), os.environ.get('IMGFLIP_PASSWORD'))
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

### detect_meme_parameters func

In [ ]:
user_message = 'I want you to create a meme, something like drake dislike, uhmm... "AI Chatbots with text inputs" and on the bottom side "Voice Commanded AI Agents"'
available_memes = imgflip_service.list_two_box_meme_names()
        
prompt = f"""You are a helpful assistant that extracts meme creation parameters from user messages.
Available meme templates are: {', '.join(available_memes)}

Your task is to:
1. Identify which meme template the user wants to use
2. Extract the text they want for the top and bottom of the meme
3. Return the information in a JSON format

Rules:
- If the user doesn't specify a meme template but their message implies a common meme format, suggest an appropriate template
- If you can't confidently match to a template, return null for meme_name
- The text should maintain the user's intent but can be adjusted to fit meme format
- If you can't determine top or bottom text, return null for those fields

Return format:
{{
    "meme_name": string or null,
    "top_text": string or null,
    "bottom_text": string or null
}}

Extract meme parameters from this message: {user_message}
"""

response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=os.environ.get("GROQ_MODEL_ID"),
        max_tokens=300,
        temperature=0,
        frequency_penalty=1.1,
        stream=False,
        response_format={"type": "json_object"}
        )

result = response.choices[0].message.content
parsed_result = json.loads(result)
print(parsed_result)

{'meme_name': 'Drake Hotline Bling', 'top_text': 'AI Chatbots with text inputs', 'bottom_text': 'Voice Commanded AI Agents'}


### __generate_processing_message func

In [ ]:
# """Generate an engaging response about content being processed"""
prompt = f"""Generate an engaging and concise response about meme being created and will be sended to the user's Telegram.

**Mention general right to the point thoughts about the meme details and please focus solely on the following specific details provided for the meme generation:**:
Meme name: {parsed_result['meme_name']}
Top Text: {parsed_result['top_text']}
Bottom Text: {parsed_result['bottom_text']}
Last user message: '''{user_message}'''

**The response should:**
1. Acknowledge the meme creation request
2. Shortly explain your thoughts about that meme
3. Mention that they'll receive a notification when it's ready
"""

print('\n\nprompt')
print(prompt)

response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=os.environ.get("GROQ_MODEL_ID"),
        max_tokens=192,
        temperature=0.08,
        frequency_penalty=1.1,
        stream=False)

response_text =response.choices[0].message.content
print('\n\nprocessing message')
print(response_text)



prompt
Generate an engaging and concise response about meme being created and will be sended to the user's Telegram.

**Mention general thoughts about the meme details and please focus solely on the following specific details provided for the meme generation:**:
Meme name: Drake Hotline Bling
Top Text: AI Chatbots with text inputs
Bottom Text: Voice Commanded AI Agents
Last user message: '''I want you to create a meme, something like drake dislike, uhmm... "AI Chatbots with text inputs" and on the bottom side "Voice Commanded AI Agents"'''

**The response should:**
1. Acknowledge the meme creation request
2. Explain and argument why your thoughts about that meme
3. Mention that they'll receive a notification when it's ready
4. Be enthusiastic and engaging



processing message
I've got it. Your "Drake Hotline Bling" meme is in the works. I love the idea of using this iconic meme to poke fun at the evolution of AI interactions. The contrast between "AI Chatbots with text inputs" and "

In [16]:
template = imgflip_service.find_template_by_name(parsed_result['meme_name'])
template

{'id': '181913649',
 'name': 'Drake Hotline Bling',
 'url': 'https://i.imgflip.com/30b1gx.jpg',
 'width': 1200,
 'height': 1200,
 'box_count': 2,
 'captions': 1348750}

In [18]:
imgflip_service.create_meme(template['id'], parsed_result['top_text'], parsed_result['bottom_text'])

{'url': 'https://i.imgflip.com/9d2ic5.jpg',
 'page_url': 'https://imgflip.com/i/9d2ic5'}